This Notebook was inspired by <https://stackoverflow.com/questions/69224096/swrl-rules-in-owl-2>. It is an (incomplete) example of how to model the relations *has_brother* and *has_uncle*. 

**Notes**:

One might think `has_brother` is

- symmetric, i.e. `has_brother(A, B)` ⇒ `has_brother(B, A)`
- transitive, i.e. `has_brother(A, B)` + `has_brother(B, C)` ⇒ `has_brother(A, C)`
- irreflexive, i.e. no one is his own brother.

However, transitivity only holds if the unique name assumption holds. Otherwise A could be the same individual as C and this conflicts irreflexivity. Thus I used a rule for this kind of "weak transitivity".

In [1]:
import time

print(time.ctime())
%load_ext nb_black
# enable special comments like `##:`
%load_ext ipydex.displaytools

Sat Oct  2 18:09:53 2021


<IPython.core.display.Javascript object>

In [2]:
import owlready2 as owl

onto = owl.get_ontology("http://test.org/onto.owl")

with onto:
    class Person(owl.Thing):
        pass

    class has_brother(owl.ObjectProperty, owl.SymmetricProperty, owl.IrreflexiveProperty):
        domain = [Person]
        range = [Person]
    
    class has_child(Person >> Person):
        pass
    
    class has_uncle(Person >> Person):
        pass

    rule1 = owl.Imp()
    rule1.set_as_rule(
        "has_brother(?p, ?b), has_child(?p, ?c) -> has_uncle(?c, ?b)"
    )

    # This rule gives "irreflexive transitivity",
    # i.e. transitivity, as long it does not lead to has_brother(?a, ?a)"
    rule2 = owl.Imp()
    rule2.set_as_rule(
        "has_brother(?a, ?b), has_brother(?b, ?c), differentFrom(?a, ?c) -> has_brother(?a, ?c)"
    )
    
david = Person("David")
john = Person("John")
pete = Person("Pete")
anna = Person("Anna")
simon = Person("Simon")

owl.AllDifferent([david, john, pete, anna, simon])

david.has_brother.extend([john, pete])

john.has_child.append(anna)
pete.has_child.append(simon)

print("Uncles of Anna:", anna.has_uncle) # -> []
print("Uncles of Simon:", simon.has_uncle) # -> []
owl.sync_reasoner(infer_property_values=True, debug=False)
print("Uncles of Anna:", anna.has_uncle) # -> [onto.Pete, onto.David]
print("Uncles of Simon:", simon.has_uncle) # -> [onto.John, onto.David]

Uncles of Anna: []
Uncles of Simon: []
Uncles of Anna: [onto.Pete, onto.David]
Uncles of Simon: [onto.John, onto.David]


<IPython.core.display.Javascript object>

In [3]:
# Some more information

david.has_brother ##:
pete.has_brother ##:
john.has_brother ##:

(david.has_brother) := [onto.John, onto.Pete]

---

(pete.has_brother) := [onto.David, onto.John]

---

(john.has_brother) := [onto.David, onto.Pete]

---

<IPython.core.display.Javascript object>